<a href="https://colab.research.google.com/github/AlexUmnov/genai_course/blob/main/week1_llm_api/homework_student.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task 1. Book summarisation

3 points

LLMSs are a great tool for summarization, but it is able to process only relatively short texts, as prescribed by the MAX TOKENS restrictions of the models. So, what if we want to summarize a whole book? Let's try to make a workaround for this.

To test our solutions we will be using CMU dataset for book summarization. Let's start with downloading a sample of book dataset from huggingface

```
@article{kryscinski2021booksum,
      title={BookSum: A Collection of Datasets for Long-form Narrative Summarization},
      author={Wojciech Kry{\'s}ci{\'n}ski and Nazneen Rajani and Divyansh Agarwal and Caiming Xiong and Dragomir Radev},
      year={2021},
      eprint={2105.08209},
      archivePrefix={arXiv},
      primaryClass={cs.CL}
}
```

In [ ]:
!curl -X GET \
     "https://datasets-server.huggingface.co/rows?dataset=kmfoda%2Fbooksum&config=default&split=train&offset=0&limit=100" > book_sum_dataset.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 3771k  100 3771k    0     0  4827k      0 --:--:-- --:--:-- --:--:-- 4828k


In [ ]:
import json

book_dataset = json.load(open("book_sum_dataset.txt"))

Let's look at one row of our database. Beware: a large text will get printed.

In [ ]:
book_dataset['rows'][0]

{'row_idx': 0,
 'row': {'bid': 27681,
  'is_aggregate': True,
  'source': 'cliffnotes',
  'chapter_path': 'all_chapterized_books/27681-chapters/chapters_1_to_2.txt',
  'summary_path': 'finished_summaries/cliffnotes/The Last of the Mohicans/section_1_part_0.txt',
  'book_id': 'The Last of the Mohicans.chapters 1-2',
  'summary_id': 'chapters 1-2',
  'content': None,
  'summary': '{"name": "Chapters 1-2", "url": "https://web.archive.org/web/20201101053205/https://www.cliffsnotes.com/literature/l/the-last-of-the-mohicans/summary-and-analysis/chapters-12", "summary": "Before any characters appear, the time and geography are made clear. Though it is the last war that England and France waged for a country that neither would retain, the wilderness between the forces still has to be overcome first. Thus it is in 1757, in the New York area between the head waters of the Hudson River and Lake George to the north. Because only two years earlier General Braddock was disgracefully routed by a hand

In [ ]:
len(book_dataset['rows'][0]['row']['chapter'])

40844

As you can see the chapters are pretty long. Let's see how many tokens we have in those chapters. Based on instruction from OpenAI we need a package `tiktoken`

We'll be following this instruction:

https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb

In [ ]:
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 12.7 MB/s eta 0:00:00


In [ ]:
import tiktoken
encoder = tiktoken.encoding_for_model("gpt-3.5-turbo")

In [ ]:
encoder.encode("Hello World!")

[9906, 4435, 0]

Please write a small function to count tokens:

In [ ]:
def count_chatgpt_tokens(text: str, tokenizer: tiktoken.Encoding) -> int:
    pass

In [ ]:
def count_chatgpt_tokens(text: str, tokenizer: tiktoken.Encoding) -> int:
    return len(encoder.encode(text))

Let's check how many tokens are there in a very simple string:

In [ ]:
count_chatgpt_tokens("Hello world!", tokenizer=encoder)

3

Now that we have this function, let's fing the maximum token length of a chapter?

In [ ]:
max(
    count_chatgpt_tokens(row['row']['chapter'], encoder)
    for row in
    book_dataset['rows']
)

13237

As we can see at the api reference page, `gpt-3.5-turbo ` has only 4096 tokens context length. Our chapters are longer then that. We could still use `pt-3.5-turbo-16k`, but it's easy to imagine texts that are longer than this, so let's learn how to do it with a smaller-context LLM.

An obvious way to cope with the problem is to:
1. Split the text into chunks of sentences that can fit into the context window.
2. Summarize each of the chunks.
3. Concatenate all the summaries. If the total length is still too big, repeat the steps 1 and 2 until it's ok.
4. Summarize the concatenations of the summaries.

## Your task

Write a function

```summarize_long_text_with_chatgpt(chapter: str) -> str```

implementing the above method of summarization.

Don't forget to log the lengths each iteration to see how much texts shrink.

For a given example please analyse your intermediate and final results. Is it indeed a good summary of the text?

**Hints and suggestions**:
- Keep in mind, that MAX TOKENS restrictions takes into account both request and model answer. So, you also need to leave some tokens for a response. So we'd suggest using at least 2:1 token ratio for chapter and summary.
- You can control the length of the summary with prompts.
- If you just use `split(".")`, you won't get a proper splitting into sentences. Luckily we have convenient Python libraries for text processing. We recommend using `sent_tokenize` from the `nltk` library. You can also try splitting the text into chunks of paragraphs instead.
- It's difficult to measure the quality of summarization, but please analyze at lease two examples. Are the summaries coherent?
- If you need inspiration in prompt building, take a look at this [paper](https://arxiv.org/pdf/2312.16171v1.pdf)

**Bonus parts:**

- Summarized text often starts with something like "This text is about", and after merging the partial summaries you'll probably have things like that all over the text. You may wish to get rid of such introductory phrases either by tuning a prompt or by post editing.

In [ ]:
!pip install nltk openai

In [ ]:
def summarize_long_text_with_chatgpt(chapter: str) -> str:
    pass

In [ ]:
sample_chapter = book_dataset['rows'][0]['row']['chapter']
summarized_chapter = summarize_long_text_with_chatgpt(sample_chapter)
print(summarized_chapter[:200])
print(len(summarized_chapter))
assert len(summarized_chapter) < len(sample_chapter) // 2

Iteration 0 length: 40844
Running iteration 0
The text describes the setting of the colonial wars in North America, focusing on the struggles and dangers faced by both French and English forces as they navigated the wilderness to engage in battle
1894


To submit the task for an automatic check, you need to generate summaries for first 10 items of this dataset and place them in a file with out predefined format

In [ ]:
with open("summarisation_result.json", 'w') as output_file:
    for row in book_dataset['rows'][:5]:
        chapter = row['row']['chapter']
        summary = summarize_long_text_with_chatgpt(chapter)
        output_file.write(
            json.dumps(
                {
                    "summary": summary,
                    "row_ids": row['row_idx']
                }
            ) + "\n"
        )

Iteration 0 length: 40844
Running iteration 0
Iteration 0 length: 18905
Running iteration 0
Iteration 0 length: 18724
Running iteration 0
Iteration 0 length: 20181
Running iteration 0
Iteration 0 length: 24398
Running iteration 0


In [ ]:
chapter_entities = [
    ["New York", "Braddock", "Magua", "Heyward"],
    ["Hawkeye", "Chingachgook", "Mohican", "Mengwe", "Iroquois", "Unca"],
    ["William Henry", "Gamut", "Heyward", "Magua", "Hawkeye"],
    ["Magua", "Heyward", "Chingachgook", "Mohican", "Gamut"],
    ["Heyward", "Edward", "Gamut", "Unca", "Cora", "Alice", "Munro"]
]

import json

def check_summarisation_results(result_path):
    results = []
    with open(result_path, 'r') as results_file:
        for line in results_file:
            results.append(json.loads(line))

    successfull_summaries = 0
    for result, entities in zip(results, chapter_entities):
        summary = result["summary"]
        found_summaries = sum(
            1 for entity in entities if entity in summary
        )
        if found_summaries >= 2:
            successfull_summaries += 1
    if successfull_summaries >= 3:
        return True
    else:
        return False


In [ ]:
check_summarisation_results("summarisation_result.json")

True

# Task 2. Extracting information with LLMs

4 points

At the practice session we were usually happy if we got something coherent. However, in real applications we often need to obtain concrete answers. Let's explore how to do it with LLMs.

Let's imagine that you work for a marketing agency, and you need to gather analytics about the passing events dedicated to AI and Machine Learning. For that, you need to process press releases and extract:
- Event name,
- Event date,
- Number of participants,
- Number of speakers,
- Attendance price.

Of course, you can do it manually, but it's much more fun to use Generative AI! So, your task will be to write a function that does this with only one request to OpenAI API.

Below there is an example of a press release (generated by ChatGPT, of course, so that both the event and the personae are fictional). All of them are in the press_releases.zip archive in the hometask week 1 folder.

<blockquote>
<p>PRESS RELEASE

InnovAI Summit 2023: A Glimpse into the Future of Artificial Intelligence</p>

City of Virtue, Cyberspace - November 8, 2023 - The most anticipated event of the year, InnovAI Summit 2023, successfully concluded last weekend, on November 5, 2023. Held in the state-of-the-art VirtuTech Arena, the summit saw a massive turnout of over 3,500 participants, from brilliant AI enthusiasts and researchers to pioneers in the field.

Esteemed speakers took to the stage to shed light on the latest breakthroughs, practical implementations, and ethical considerations in AI. Dr. Evelyn Quantum, renowned for her groundbreaking work on Quantum Machine Learning, emphasized the importance of this merger and how it's revolutionizing computing as we know it. Another keynote came from Prof. Leo Nexus, whose current project 'AI for Sustainability' highlights the symbiotic relationship between nature and machine, aiming to use AI in restoring our planet's ecosystems.

This year's panel discussion, moderated by the talented Dr. Ada Neura, featured lively debates on the limits of AI in creative arts. Renowned digital artist, Felix Vortex, showcased how he uses generative adversarial networks to create surreal art pieces, while bestselling author, Iris Loom, explained her experiments with AI-assisted story crafting.

Among other highlights were hands-on workshops, interactive Q&A sessions, and an 'AI & Ethics' debate which was particularly well-received, emphasizing the need for transparency and fairness in AI models. An exclusive 'Start-up Alley' allowed budding entrepreneurs to showcase their innovations, gaining attention from global venture capitalists and media.

The event wrapped up with an announcement for InnovAI Summit 2024, set to be even grander. Participants left with a renewed enthusiasm for the vast possibilities that the AI and ML world promises.

For media inquiries, please contact:
Jane Cipher
Director of Communications, InnovAI Summit
Email: jane.cipher@innovai.org
Phone: +123-4567-8910</p>
</blockquote>

More specifically, you should write a function

```python
parse_press_release(pr: str) -> dict
```

where the output should be in the format

```python
{
  name: 'InnovAI Summit 2023',
  date: '08.11.2023',
  n_participants: 3500,
  n_speakers: 4,
  price:
}
```

If any of the four characteristics is not mentioned in the text, put `None` in the respective field.

At the end, calculate the statistics of right answers and analyse what kind of mistakes you "model" makes the most.

**Hints and suggestions:**
- It's gonna be more convenient to experiment in OpenAI chat interface https://chat.openai.com/. Plus this doesn't cost API requests money.
- You need to be very accurate with what you want from the model.
- It will help if you specify in the prompt that the output should be in JSON format, this way you will spend less time parsing the output. You can also try using forced structured output vs just mentioning the format.
- Please be careful with the details. For example, Jane Cipher in the text above is not a speaker and shouldn't be counter as such (how to get rid of a contact person?). Also pay attention to the date format,
- If the model is too wilful with the output format, don't hesitate to show some examples. Decreasing the temperature of predictions can help reduce the creativity of the answer, which is what we want for such task.
- Debugging an LLM-powered application may become a tough business. When you think that you've polished it, an LLM can still surprise you. So, we don't expect 100% accuracy in this task, but we expect that you do your best to achieve high quality results.

In [ ]:
press_release = """PRESS RELEASE

InnovAI Summit 2023: A Glimpse into the Future of Artificial Intelligence

City of Virtue, Cyberspace - November 8, 2023 - The most anticipated event of the year, InnovAI Summit 2023, successfully concluded last weekend, on November 5, 2023. Held in the state-of-the-art VirtuTech Arena, the summit saw a massive turnout of over 3,500 participants, from brilliant AI enthusiasts and researchers to pioneers in the field.

Esteemed speakers took to the stage to shed light on the latest breakthroughs, practical implementations, and ethical considerations in AI. Dr. Evelyn Quantum, renowned for her groundbreaking work on Quantum Machine Learning, emphasized the importance of this merger and how it's revolutionizing computing as we know it. Another keynote came from Prof. Leo Nexus, whose current project 'AI for Sustainability' highlights the symbiotic relationship between nature and machine, aiming to use AI in restoring our planet's ecosystems.

This year's panel discussion, moderated by the talented Dr. Ada Neura, featured lively debates on the limits of AI in creative arts. Renowned digital artist, Felix Vortex, showcased how he uses generative adversarial networks to create surreal art pieces, while bestselling author, Iris Loom, explained her experiments with AI-assisted story crafting.

Among other highlights were hands-on workshops, interactive Q&A sessions, and an 'AI & Ethics' debate which was particularly well-received, emphasizing the need for transparency and fairness in AI models. An exclusive 'Start-up Alley' allowed budding entrepreneurs to showcase their innovations, gaining attention from global venture capitalists and media.

The event wrapped up with an announcement for InnovAI Summit 2024, set to be even grander. Participants left with a renewed enthusiasm for the vast possibilities that the AI and ML world promises.

For media inquiries, please contact: Jane Cipher Director of Communications, InnovAI Summit Email: jane.cipher@innovai.org Phone: +123-4567-8910"""

In [ ]:
def parse_press_release(pr: str) -> dict:
    pass

In [ ]:
parse_press_release(press_release)

{'name': 'InnovAI Summit 2023',
 'date': '05.11.2023',
 'n_participants': 3500,
 'n_speakers': 4,
 'price': 'None'}

###Testing
We prepared a small dataset for you to test your prompt on.
Provided you've written your function, try running the following code.
At the end you also have an opportunity to look at the results in a table side-by-side in `with_results.csv`.
Your goal is to get at least 60% accuracy, or 26 fields right.

In [ ]:
!pip install --upgrade gdown
!gdown -O press_release_extraction.csv https://docs.google.com/spreadsheets/d/15IGdc3MV8864lxrLxsug0Ij480p76T1EAwBM7WGT_OI/export?format=csv

/usr/local/lib/python3.10/dist-packages/gdown/parse_url.py:48: UserWarning: You specified a Google Drive link that is not the correct link to download a file. You might want to try `--fuzzy` option or the following url: https://drive.google.com/uc?id=None
  warnings.warn(
Downloading...
From: https://docs.google.com/spreadsheets/d/15IGdc3MV8864lxrLxsug0Ij480p76T1EAwBM7WGT_OI/export?format=csv
To: /content/press_release_extraction.csv
16.0kB [00:00, 25.0MB/s]


In [ ]:
import pandas
pr_df = pandas.read_csv("press_release_extraction.csv")
pr_df.head()

pr_text  \
0  InnovAI Summit 2023: A Glimpse into the Future...   
1  Press Dispatch: 'Artificial Mariners: Navigati...   
2  FOR IMMEDIATE RELEASE\n\nAI Innovators Convene...   
3  Press Release: Cutting-Edge Innovations Debute...   
4  Press Release: Innovative Minds Gather at "AI ...   

                                           pr_parsed  
0  {\n  "name": "InnovAI Summit 2023",\n  "date":...  
1  {"name": "Artificial Mariners: Navigatin' the ...  
2  {"name": "Annual Machine Learning Symposium 20...  
3  {"name": "AI Advancements Summit 2023",\n "dat...  
4  {"name": "AI Horizon 2023",\n "date": "15.10.2...

In [ ]:
pr_df.pr_parsed[0]

'{\n  "name": "InnovAI Summit 2023",\n  "date": "05.11.2023",\n  "n_participants": 3500,\n  "n_speakers": 4,\n  "price": "None"\n}'

In [ ]:
import json

parsed_list = []
fields = {
    "name": str,
    "date": str,
    "n_speakers": int,
    "n_participants": int,
    "price": str
}
correct_fields = 0
for row in pr_df.itertuples():
    parsed_release = parse_press_release(row.pr_text)
    parsed_list.append(json.dumps(parsed_release, indent=4))
    golden = json.loads(row.pr_parsed)
    for field, field_type in fields.items():
        golden_field = golden[field]
        parsed_field = parsed_release.get(field)
        try:
            parsed_field = field_type(parsed_field)
        except (ValueError, TypeError):
            pass
        if golden_field == parsed_field:
            correct_fields += 1
        else:
            print(f"For {golden['name']} {field} {parsed_release.get(field)} doesn't seem the same as {golden[field]}")

print(correct_fields)

For Annual Machine Learning Symposium 2023 n_speakers 3 doesn't seem the same as 4
For AI Horizon 2023 n_speakers 0 doesn't seem the same as None
33


In [ ]:
pr_df['results'] = parsed_list
pr_df.to_csv("with_results.csv")

# Task 2.1 Make sure to try both Anthropic and OpenAI

It's important here to compare different models. We suggest you try at least one model from OpenAI and one model from Anthropic. However, if you'd like to, you can also try different tiers of models (i.e. gpt4o vs gpt4o-mini).

# Task 2.2 Advanced structured output

As we've seen in the seminar, ChatGPT supports outputting in a specific format of a pydantic structure. Because in this case, our output has predefined fields, we can make use of it. Define the structure, set up the output format and measure the quality.

Here's an example of a pydantic model for the output with a simple validator:

In [ ]:
from typing import List, Any
from pydantic import BaseModel, field_validator

class PressRelease(BaseModel):
    n_speakers: int

    @field_validator('n_speakers')
    @classmethod
    def speaker_number_positive(cls, value: int) -> int:
        if value <= 0:
            raise ValueError('Number of speakers should be negative')
        return value


You task here is to finish the model to incorporate all the required fields.

Keep in mind, that for each python type like `int` or `str`, you can make it `Optional`, which then will also allow it to be `None`. However, depending on how you prompted your model you might receive "None" (string) instead of `None`. In that case you can have a choice of types, like `int | str`.

You also need to write a validator for at least the `date` field so that both DD.MM.YYYY and DD.MM.YYYY-DD.MM.YYYY formats were acceptable.


Optionally try to think of some more ways you can validate the output. Some of the examples are:
- The price is in format CUR #NUMBER, i.e. EUR 100;
- Check that the date mentioned in the name of the event is consistent with the date in `date` field.


Then run the inference with structured input tied to that model. Make sure to incorporate retries based on Validation Errors in case the LLM didn't fill the fields correctly.

To do that you can catch `pydantic.ValidationError`

# Task 3: Bonus task.

1 point

It's quite important to understand the current limitations of the technology. As for LLMs, there are still plenty of weak spots. They can struggle even with such an example:

In [ ]:
sentence = "How many tokens are in this sentence?"
print(f"Actual token count: {len(encoder.encode(sentence))}")
print(f"ChatGPT thinks it's: {get_chatgpt_answer(sentence)}")

Or with this one:

In [ ]:
# example
sentence = "Reverse this sentence character by character"
reversed_sentence = get_chatgpt_answer(sentence)
print(reversed_sentence[::-1])

### LLM doing math

Math is also not too easy for LLMs which are getting better at counting and mathematical reasoning thanks to chain-of-thought generation, but can still struggle with symbolic algebra.

Let us look at an example:

In [ ]:
sentence = "Let's define a mathematical operation x * y := xy + x + y. Is it associative?"
print(get_chatgpt_answer(sentence))

**The solution by ChatGPT-4**

The

To determine if the operation $*$ is associative, we need to check if:

$$(a * b) * c = a * (b * c)$$

for all real numbers $a$, $b$, and $c$. If this equality holds for all real numbers, then the operation is associative.

Given the operation $x∗y:=xy+x+y$:

Calculating $(a∗b)∗c$:

First,

$$a∗b=ab+a+b.$$

Using this result:

$$(a∗b)∗c=(ab+a+b)∗c=$$
$$=(ab+a+b)c+(ab+a+b)+c=$$
$$=abc+ac+bc+ab+a+b+c$$

Calculating $a∗(b∗c)$:

First,

$$b∗c=bc+b+c.$$

Using this result:

$$a∗(b∗c)=a∗(bc+b+c)=$$
$$=a(bc+b+c)+a+bc+b+c=$$
$$=abc+ab+ac+a+bc+b+c$$

Comparing the two results:

$$(a∗b)∗c=abc+ac+bc+ab+a+b+c$$

$$a∗(b∗c)=abc+ab+ac+a+bc+b+c$$

The two expressions are not equal for all real numbers $a$, $b$, and $c$. Therefore, the operation $*$ is not associative.

**End**

Let's analyze this solution. You can see that ChatGPT knows definitions and does well with logic, but fails at the very last stage where it can't understand that

$$abc+ac+bc+ab+a+b+c$$

and

$$abc+ab+ac+a+bc+b+c$$

is the same expression with permuted summands.

## Task 3*

Find out what is it you are good at, but ChatGPT cannot do. Please try to be objective. Bonus points for analyzing stability of the failures of ChatGPT and their dependence on prompt formulation.